In [40]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
from tabulate import tabulate
import os
import time 

In [ ]:
d = webdriver.Chrome()

In [2]:
def weather_correct(data):
    weather_dict={}
    weather_dict["Time"]=data.iloc[0,0]
    data.drop(columns=["Unnamed: 1","Weather","Wind","Unnamed: 5","Visibility"],inplace=True)
    data.fillna(method="ffill",inplace=True)
    data.dropna(how="any",inplace=True)
    data.reset_index(inplace=True)
    temp=0
    humidity=0
    pressure=0
    for f in range(data.shape[0]):
        pattern = r'([+-]?\d+(\.\d+)*)\s?°([CcFf])'
        a_temp=re.findall(pattern,data["Temp"][f])
        temp=float(a_temp[0][0])+temp
        a_humidity= data["Humidity"][f].split("%")
        humidity=float(a_humidity[0])+humidity
        a_pressure=data["Barometer"][f].split(" ")
        pressure=float(a_pressure[0])+pressure
    temp=temp/len(data)
    humidity=humidity/len(data)
    pressure=pressure/len(data)
    weather_dict["Temp"]=temp
    weather_dict["Humidity"]=humidity
    weather_dict["Pressure"]=pressure
    df=pd.DataFrame(columns=["Time","Temp","Humidity","Pressure"])
    df=df.append(weather_dict,ignore_index=True)
    return df

In [6]:
def format_df(data):
    a_temp=list(data["Temp"])
    len_df=data.shape[0]
    for f in range(len_df):
        a_temp

In [11]:
def table_content(a):
    soup_level1=BeautifulSoup(a, 'lxml')
    table = soup_level1.find_all('table')[1]
    df = pd.read_html(str(table),header=0)
    df = pd.read_html(str(table),header=1)
    df=df[0]
    df=df.drop(df.index[len(df)-1])
    
    return df

In [7]:
def link_change(url):
    d = webdriver.Chrome()
    d.get(url)
    df=[]
    q=d.find_element_by_xpath("/html/body/div[1]/div[7]/article/section[1]/div[5]/div[3]").text
    x=len(q.split(" "))
    for f in range(1,x):
        d.find_element_by_xpath("/html/body/div[1]/div[7]/article/section[1]/div[5]/div[3]/a["+str(f)+"]").click()
        time.sleep(8)
        d_name="df"+test_list[f]
        d_name=table_content(d.page_source)
        d_name=weather_correct(d_name)
        df.append(d_name)
    df_final=pd.concat(df,ignore_index =True)
    return df_final

In [16]:
df_year_wise_total=[]
for q in range(2019,2020):
    df_month_wise_total=[]
    for f in range(1,13):
        url="https://www.timeanddate.com/weather/india/kota/historic?month="+str(f)+"&year="+str(q)
        time.sleep(10)
        df_month_wise="df1"+test_list[f]
        df_month_wise=link_change(url)
        name="weather"+str(f)+str(q)+".csv"
        df_month_wise_total.append(df_month_wise)
        df_month_wise.to_csv(name)
    df_year_wise="df2"+test_list[q%2000]
    df_year_wise=pd.concat(df_month_wise_total,ignore_index =True)
    df_year_wise_total.append(df_year_wise)
    name_1="weather"+str(q)+".csv"
    df_year_wise.to_csv(name_1)
df_year_wise=pd.concat(df_year_wise_total,ignore_index =True) 

In [14]:
df_year_wise.to_csv("finallweather.csv")